In [ ]:
import numpy as np
import pandas as pd
from gpcam.gp_optimizer import GPOptimizer
import matplotlib.pyplot as plt
import mpltern
from matplotlib.gridspec import GridSpec
from matplotlib.patches import ArrowStyle, FancyArrowPatch

In [ ]:
def add_arrows_to_ternary_plot(label_top, label_left, label_right, ax=None):
    ax = ax or plt.gca()
    arrowstyle = ArrowStyle("simple", head_length=10, head_width=5)
    kwargs_arrow = {
        "transform": ax.transAxes,  # Used with ``ax.transAxesProjection``
        "arrowstyle": arrowstyle,
        "linewidth": 1,
        "clip_on": False,  # To plot arrows outside triangle
        "zorder": -10,  # Very low value not to hide e.g. tick labels.
    }

    # Start of arrows in barycentric coordinates.
    ta = np.array([0.0, -0.1, 1.1])
    la = np.array([1.1, 0.0, -0.1])
    ra = np.array([-0.1, 1.1, 0.0])

    # End of arrows in barycentric coordinates.
    tb = np.array([1.0, -0.1, 0.1])
    lb = np.array([0.1, 1.0, -0.1])
    rb = np.array([-0.1, 0.1, 1.0])

    # This transforms the above barycentric coordinates to the original Axes
    # coordinates. In combination with ``ax.transAxes``, we can plot arrows fixed
    # to the Axes coordinates.
    f = ax.transAxesProjection.transform

    tarrow = FancyArrowPatch(f(ta), f(tb), ec="k", fc="k", **kwargs_arrow)
    larrow = FancyArrowPatch(f(la), f(lb), ec="k", fc="k", **kwargs_arrow)
    rarrow = FancyArrowPatch(f(ra), f(rb), ec="k", fc="k", **kwargs_arrow)
    ax.add_patch(tarrow)
    ax.add_patch(larrow)
    ax.add_patch(rarrow)

    # To put the axis-labels at the positions consistent with the arrows above, it
    # may be better to put the axis-label-text directly as follows rather than
    # using e.g.  ax.set_tlabel.
    kwargs_label = {
        "transform": ax.transTernaryAxes,
        "backgroundcolor": "w",
        "ha": "center",
        "va": "center",
        "rotation_mode": "anchor",
        "zorder": -9,  # A bit higher on arrows, but still lower than others.
    }

    # Put axis-labels on the midpoints of arrows.
    tpos = (ta + tb) * 0.5
    lpos = (la + lb) * 0.5
    rpos = (ra + rb) * 0.5

    ax.text(*tpos, label_top, c="k", rotation=-60, **kwargs_label)
    ax.text(*lpos, label_left, c="k", rotation=60, **kwargs_label)
    ax.text(*rpos, label_right, c="k", rotation=0, **kwargs_label)

# Plotting of measurements taken so far

In [ ]:
df = pd.read_csv(
    "/Users/wiebke/Documents/Data/saxs-waxs-samples/BCP_Mixing/2024_10_17-23_58k-34k_mixture/2024_10_17-23_58k-34k_mixture.csv"
)
additive = df["Fraction Additive"].values
s2vp34k = df["Fraction 34k"].values
s2vp58k = df["Fraction 58k"].values
fwhms = df["Fwhm"].values
peak_positions = df["Peak Position"].values

In [ ]:
df

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 6), subplot_kw={"projection": "ternary"})

material_top = "Additive"
material_left = "S2VP58k"
material_right = "S2VP34k"

# Plot 1: Colored by s2vp35k_ratio
sc1 = axs[0].scatter(
    additive,
    s2vp58k,
    s2vp34k,
    c=peak_positions,
    cmap="viridis",
    vmin=0.1 * 2 * np.pi / 42,  # 42nm domain spacing
    vmax=0.1 * 2 * np.pi / 25,  # 25nm domain spacing
)
axs[0].set_title(f"Peak position")
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[0]
)
fig.colorbar(sc1, ax=axs[0], label=f"peak position", shrink=0.5)
axs[0].grid()

# Plot 2: Colored by fwhm
sc2 = axs[1].scatter(
    additive, s2vp58k, s2vp34k, c=fwhms, cmap="inferno", vmin=0.003, vmax=0.006
)
axs[1].set_title(f"Fwhm")
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[1]
)
fig.colorbar(sc2, ax=axs[1], label="fwhm", shrink=0.5)
axs[1].grid()

# Preperation of data for Gaussian Process

In [ ]:
domain_spacing = 30  # nm
q_target = 0.1 * 2 * np.pi / domain_spacing
peak_pos_min = 0.1 * 2 * np.pi / 42
peak_pos_max = 0.1 * 2 * np.pi / 25
max_peak_dist = max(np.abs(peak_pos_min - q_target), np.abs(peak_pos_max - q_target))
print(max_peak_dist)

In [ ]:
x_data = np.stack([s2vp58k, s2vp34k], axis=1)
x_data.shape

y_data_peaks_only = peak_positions - q_target
y_data_fwhm_only = fwhms - 0.002
y_data_mixed = 0.2 * abs(y_data_peaks_only) / (max_peak_dist) + 0.8 * (
    abs(y_data_fwhm_only) / (0.003) - 0.003
)

In [ ]:
# bound of the input space (parameters that are controlled)
bounds = np.array([[0, 1], [0, 1]])
# Set up hyperparameters for kernel
hps_bounds = np.array([[0.0001, 1], [0.0001, 1], [0.0001, 1]])
# initialize the GPOptimizer
my_gpo = GPOptimizer(x_data, peak_positions, init_hyperparameters=np.ones(3) * 0.1)
# and train it
my_gpo.train(hyperparameter_bounds=hps_bounds, max_iter=10000, method="mcmc")
print("hyperparameters after 1st training: ", my_gpo.hyperparameters)

In [ ]:
x_pred = np.random.dirichlet([1, 1, 1], size=1500)
x_pred_reduced = x_pred[:, :2]
print(x_pred.shape)
mean_calulation = my_gpo.posterior_mean(x_pred_reduced)
mean = mean_calulation["f(x)"]
variance_calculation = my_gpo.posterior_covariance(x_pred_reduced)
covariance = variance_calculation["v(x)"]

# Plot current state of the Gaussian Process

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 6), subplot_kw={"projection": "ternary"})

sc1 = axs[0].scatter(
    additive,
    s2vp58k,
    s2vp34k,
    c=[peak_positions],
    cmap="viridis",
    # vmin=0.003,
    # vmax=0.006,
)
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[0]
)
fig.colorbar(sc1, ax=axs[0], label=f"fwhm", shrink=0.5)
axs[0].grid()

levels = [0.1 * 2 * np.pi / q_target_nm for q_target_nm in [25, 27.5, 30, 32.5, 35]]
# sc2 = axs[1].tripcolor(
sc2 = axs[1].tricontour(
    x_pred[:, 2],
    x_pred[:, 0],
    x_pred[:, 1],
    mean,
    level=levels,
    # shading="gouraud",
    # rasterized=True,
    # vmin=0.003,
    # vmax=0.006,
)
axs[1].clabel(sc2)
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[1]
)
fig.colorbar(sc2, ax=axs[1], label=f"gp mean", shrink=0.5)
axs[1].grid()

sc3 = axs[2].tripcolor(
    x_pred[:, 2],
    x_pred[:, 0],
    x_pred[:, 1],
    covariance,
    cmap="inferno",
    shading="gouraud",
    rasterized=True,
)
add_arrows_to_ternary_plot(
    f"{material_top} (%)", f"{material_left} (%)", f"{material_right} (%)", ax=axs[2]
)
axs[2].grid()
fig.colorbar(sc3, ax=axs[2], label=f"gp covariance", shrink=0.5)
plt.tight_layout()

# Retrieve suggestions from the Gaussian Process

In [ ]:
from scipy.optimize import LinearConstraint

# Change the target value here:
q_target_nm = 27.5
q_target = 0.1 * 2 * np.pi / q_target_nm


def acq_func_target(x, obj):
    mean = obj.posterior_mean(x)["f(x)"]
    return -abs(mean - q_target)


def acq_func_target_uncertainty(x, obj):
    a = 3.0  # 3.0 for 95 percent confidence interval
    mean = obj.posterior_mean(x)["f(x)"]
    cov = obj.posterior_covariance(x)["v(x)"]
    return -abs(mean - q_target) * np.sqrt(cov)


A = np.array([1, 1])
# Swell ratio betwen 2.25 (1.25/1) and 6 (5/1)
lower_bound = 1 / 6
upper_bound = 1 / 2.25
lc = LinearConstraint(A, lower_bound, upper_bound)

df = pd.DataFrame(
    columns=[
        "Fraction 58k",
        "Fraction 34k",
        "Fraction Additive",
        "Swell Ratio",
        "Step 1, 58k",
        "Step 1, 34k",
    ]
)
for n in range(1, 20):
    new_x_peaks = my_gpo.ask(
        bounds,
        n=1,
        acquisition_function=acq_func_target,  # May change aquisition function here
        # pop_size=20,
        # max_iter=20,
        # tol=1e-3,
        constraints=(lc,),
    )
    fraction_58k = new_x_peaks["x"][0][0]
    fraction_34k = new_x_peaks["x"][0][1]
    new_row = pd.DataFrame(
        {
            "Fraction 58k": [fraction_58k],
            "Fraction 34k": [fraction_34k],
            "Step 1, 58k": [fraction_58k / (fraction_58k + fraction_34k)],
            "Step 1, 34k": [fraction_34k / (fraction_58k + fraction_34k)],
        },
    )
    df = pd.concat([df, new_row], ignore_index=True)

df["Fraction Additive"] = 1 - df["Fraction 58k"] - df["Fraction 34k"]
df["Swell Ratio"] = 1 + df["Fraction Additive"] / (
    df["Fraction 34k"] + df["Fraction 58k"]
)
print(df)
df.to_csv(f"suggestions_peak_position_{q_target_nm}nm_target_only.csv")